We define classes for the problem.

In [2]:
from typing import List
import numpy as np

class Car:
    def __init__(self,car_id: int, time_slots_at_charging_unit: List[int], required_energy: int)-> None :
        self.car_id = car_id
        self.time_slots_at_charging_unit = time_slots_at_charging_unit
        self.required_energy = required_energy

    def __str__(self) -> str:
        return f"Car {self.car_id}: \n" \
               f" at charging station at time slots {self.time_slots_at_charging_unit} \n "\
               f" requires {self.required_energy} energy units"
    

class ChargingUnit:
    def __init__(self, unit_name: str, num_charging_levels: int, num_time_slots: int) -> None:
        self.unit_name = unit_name
        self.num_charging_levels = num_charging_levels
        self.num_time_slots = num_time_slots
        
        self.car_to_charge = []

    def __str__(self) -> str:
        info_cars_registered = ""
        for car in self.cars_to_charge:
            info_cars_registered = info_cars_registered + " " + car.car_id
        return "Charging unit with\n" \
            " charging levels: " \
            f"{list(range(self.num_charging_levels))[1:-1]}\n" \
            " time slots: " \
            f"{list(range(self.num_time_slots))[1:-1]}\n" \
            " cars to charge:" \
            + info_cars_registered
    
    def register_car_for_charging(self, car: Car) -> None:
        if max(car.time_slots_at_charging_unit) > self.num_time_slots - 1:
            raise ValueError("From car required time slots not compatible "
            " with charging unit.")
        self.cars_to_charge.append(car)

2